In [5]:
import datetime
import time
import pandas as pd
import requests
import json
import pandas as pd
import numpy as np
from jsonpath import jsonpath
import telebot
import hmac, base64, struct, hashlib
import platform


# google验证码函数
submit_url = 'http://fundmng.bsportsadmin.com/api/manage/user/admin/login/submit'
header0 = {
    'Device_id':'1.0',
    'Os_type':'0',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
}
def get_google_code(secret):
    key = base64.b32decode(secret, True)
    msg = struct.pack(">Q", int(time.time()) // 30)
    google_code = hmac.new(key, msg, hashlib.sha1).digest()
    # 很多网上的代码不可用，就在于这儿，没有chr字符串
    o = ord(chr(google_code[19])) & 15
    # google_code = (struct.unpack(">I", google_code[o:o + 4])[0] & 0x7fffffff) % 1000000
    google_code = (struct.unpack(">I", google_code[o:o + 4])[0] & 0x7fffffff) % 1000000
    return '%06d' % google_code

# 生成验证码
print('生成google验证码。。。')
google_code = get_google_code('64ehnxj6yily5bhv23kgb62ozuh6yuu2')
data0 = {
    'username': 'Marquis',
    'password': 'qwer123456',
    'code': google_code
    }
# 获取token
print('获取token中。。。')
session0 = requests.Session()
response0  =session0.post(url=submit_url,data=data0,headers=header0)
response0.encoding = 'utf-8'
obj0 = json.loads(response0.text)
token = obj0['data']['token']

# 发送请求
print('开始发送请求....')
url = 'http://fundmng.bsportsadmin.com/api/manage/fund/recharge/record/list'
header = {
        'Device_id': '1.0',
        'Os_type': '0',
        'Sign': '525f286b21988b7c6a79f623f62f9695',
        'Timestamp': '1691886184000',
        'Token': token,
        'Uid': '690',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
        'Version': '1.0'
    }
session  = requests.Session()
dic0 = {'充值日期':"createDate",'充值单号':"orderId",'用户层级':"userLevel",'账户名':"username",'姓名':"reallyName",'VIP等级':"vipLevel",'充值金额':"amount",'到账金额':"payAmount"}
dic = {'充值日期':[],'充值单号':[],'用户层级':[],'账户名':[],'姓名':[],'VIP等级':[],'充值金额':[],'到账金额':[]}
for page in range(1,60):
    data = {
            'dateType':'1',
            'startTime':int(time.time()-(2*60*60))*1000,
            'endTime':int(time.time())*1000,
            'userType':'-1',
            'orderStatus':'0,1,2,3,4,5,6,7,8,9',
            'agentType':'-1',
            'page':page,
            'size':500,
            'paymentIdList':'1,2,3,4,5,6,7,8,9,10,11,24,25,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,2000,2001,2002,2003,2004,2005,2006,3000,3001,4000,4001,4002,4003,4004,8000,8001,8002,10000,10001',
            'coinCode':'CNY'
        }

    response = session.post(url=url,headers=header,data=data)
    response.encoding = 'utf8'

    # 提取数据

    obj = json.loads(response.text)
    for i in obj['data']['list']:
        # if (i["userLevel"] == '多次拉单不付款层级' and i["payAmount"] ==0 ):
        if (i["userLevel"] == '多次拉单不付款层级' and i["payAmount"] ==0 and i["vipLevel"] > -1):
            for k in dic:
                dic[k].append(i[dic0[k]])
# 生成结果数据
df = pd.DataFrame(dic)
df.insert(1,'时间',df['充值日期'].map(lambda x:time.strftime("%H:%M")))
df['充值日期']= df['充值日期'].map(lambda x:time.strftime("%Y-%m-%d"))
print(df)
result = df.groupby('账户名').agg({'账户名':len,'充值金额':[np.max,np.mean],'VIP等级':np.mean})
# result.reset_index(inplace=True)
result

生成google验证码。。。
获取token中。。。
开始发送请求....
          充值日期     时间                  充值单号       用户层级            账户名   姓名  \
0   2023-09-11  17:42  CZ202309111658B3CK12  多次拉单不付款层级   qq1027580306   周亚   
1   2023-09-11  17:42  CZ202309111657B3CH5O  多次拉单不付款层级   qq1027580306   周亚   
2   2023-09-11  17:42  CZ202309111634B37184  多次拉单不付款层级       hs451622   侯松   
3   2023-09-11  17:42  CZ202309111634B36Z8W  多次拉单不付款层级       hs451622   侯松   
4   2023-09-11  17:42  CZ202309111634B36VRJ  多次拉单不付款层级       hs451622   侯松   
5   2023-09-11  17:42  CZ202309111633B36RKI  多次拉单不付款层级       hs451622   侯松   
6   2023-09-11  17:42  CZ202309111632B36N0M  多次拉单不付款层级       hs451622   侯松   
7   2023-09-11  17:42  CZ202309111632B36JVD  多次拉单不付款层级       hs451622   侯松   
8   2023-09-11  17:42  CZ202309111631B36ECK  多次拉单不付款层级       hs451622   侯松   
9   2023-09-11  17:42  CZ202309111631B369LY  多次拉单不付款层级       hs451622   侯松   
10  2023-09-11  17:42  CZ202309111630B35TE3  多次拉单不付款层级       hs451622   侯松   
11  2023-09-11  17:42  CZ2

账户名     充值金额          VIP等级
              len      max     mean  mean
账户名                                      
aa13265755591   1   1007.0   1007.0   0.0
hs451622       10   1000.0   1000.0   0.0
qq1027580306    2   2000.0   1500.0   0.0
wenqi1995       1  29000.0  29000.0   0.0
yy13484173836   3   1000.0   1000.0   0.0

In [8]:
vip_df = pd.read_csv(r'C:\Users\User\IdeaProjects\project1\Bsport\充值拉单预警\vip.csv')

,会员账号,VIP等级
0,thdh445e5,1
1,Hoangphu626163,1
2,Hoanghoa89,1
3,clone267,1
4,ytt1999,1


In [9]:
vip_df2 = vip_df.copy().set_index('会员账号')
vip_df2.head()

,VIP等级
会员账号,
thdh445e5,1
Hoangphu626163,1
Hoanghoa89,1
clone267,1
ytt1999,1


In [12]:
result.join(vip_df2)

C:\Users\User\AppData\Local\Temp\ipykernel_20708\2179653192.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  result.join(vip_df2)


,"(账户名, len)","(充值金额, max)","(充值金额, mean)","(VIP等级, mean)",VIP等级
aa13265755591,1,1007.0,1007.0,0.0,2
aa13265755591,1,1007.0,1007.0,0.0,2
aa13265755591,1,1007.0,1007.0,0.0,1
hs451622,10,1000.0,1000.0,0.0,5
hs451622,10,1000.0,1000.0,0.0,4
hs451622,10,1000.0,1000.0,0.0,4
qq1027580306,2,2000.0,1500.0,0.0,4
qq1027580306,2,2000.0,1500.0,0.0,3
qq1027580306,2,2000.0,1500.0,0.0,2
qq1027580306,2,2000.0,1500.0,0.0,1


In [18]:
tem = vip_df.groupby('会员账号').agg({'会员账号':len})
tem.loc[tem['会员账号']>1,:]

,会员账号
会员账号,
00000000z,2
000125aa,4
000310555lzf,4
000369kk,4
000628yy,3
...,...
zzzzzz172839,3
zzzzzz36,3
zzzzzz654321,3


In [ ]:
import datetime
import time
import pandas as pd
import requests
import json
import pandas as pd
import numpy as np
from jsonpath import jsonpath
import telebot
import hmac, base64, struct, hashlib
import platform


# google验证码函数
submit_url = 'http://fundmng.bsportsadmin.com/api/manage/user/admin/login/submit'
header0 = {
    'Device_id':'1.0',
    'Os_type':'0',
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
}
def get_google_code(secret):
    key = base64.b32decode(secret, True)
    msg = struct.pack(">Q", int(time.time()) // 30)
    google_code = hmac.new(key, msg, hashlib.sha1).digest()
    # 很多网上的代码不可用，就在于这儿，没有chr字符串
    o = ord(chr(google_code[19])) & 15
    # google_code = (struct.unpack(">I", google_code[o:o + 4])[0] & 0x7fffffff) % 1000000
    google_code = (struct.unpack(">I", google_code[o:o + 4])[0] & 0x7fffffff) % 1000000
    return '%06d' % google_code


def job():
    # 生成验证码
    print('生成google验证码。。。')
    google_code = get_google_code('64ehnxj6yily5bhv23kgb62ozuh6yuu2')
    data0 = {
        'username': 'Marquis',
        'password': 'qwer123456',
        'code': google_code
    }
    # 获取token
    print('获取token中。。。')
    session0 = requests.Session()
    response0  =session0.post(url=submit_url,data=data0,headers=header0)
    response0.encoding = 'utf-8'
    obj0 = json.loads(response0.text)
    token = obj0['data']['token']

    # 发送请求
    print('开始发送请求....')
    url = 'http://fundmng.bsportsadmin.com/api/manage/fund/recharge/record/list'
    header = {
        'Device_id': '1.0',
        'Os_type': '0',
        'Sign': '525f286b21988b7c6a79f623f62f9695',
        'Timestamp': '1691886184000',
        'Token': token,
        'Uid': '690',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
        'Version': '1.0'
    }
    session  = requests.Session()
    dic0 = {'充值日期':"createDate",'充值单号':"orderId",'用户层级':"userLevel",'账户名':"username",'姓名':"reallyName",'VIP等级':"vipLevel",'充值金额':"amount",'到账金额':"payAmount"}
    dic = {'充值日期':[],'充值单号':[],'用户层级':[],'账户名':[],'姓名':[],'VIP等级':[],'充值金额':[],'到账金额':[]}
    for page in range(1,60):
        data = {
            'dateType':'1',
            'startTime':int(time.time()-(2*60*60))*1000,
            'endTime':int(time.time())*1000,
            'userType':'-1',
            'orderStatus':'0,1,2,3,4,5,6,7,8,9',
            'agentType':'-1',
            'page':page,
            'size':500,
            'paymentIdList':'1,2,3,4,5,6,7,8,9,10,11,24,25,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,2000,2001,2002,2003,2004,2005,2006,3000,3001,4000,4001,4002,4003,4004,8000,8001,8002,10000,10001',
            'coinCode':'CNY'
        }

        response = session.post(url=url,headers=header,data=data)
        response.encoding = 'utf8'

        # 提取数据

        obj = json.loads(response.text)
        for i in obj['data']['list']:
            # if (i["userLevel"] == '多次拉单不付款层级' and i["payAmount"] ==0 ):
            if (i["userLevel"] == '多次拉单不付款层级' and i["payAmount"] ==0 and i["vipLevel"] > -1):
                for k in dic:
                    dic[k].append(i[dic0[k]])
    # 生成结果数据
    df = pd.DataFrame(dic)
    df.insert(1,'时间',df['充值日期'].map(lambda x:time.strftime("%H:%M")))
    df['充值日期']= df['充值日期'].map(lambda x:time.strftime("%Y-%m-%d"))
    print(df)
    result = df.groupby('账户名').agg({'账户名':len,'充值金额':[np.max,np.mean],'VIP等级':np.mean})
    result.reset_index(inplace=True)
    result = pd.DataFrame({'账户名':result.iloc[:,0],'数量':result.iloc[:,1],'最大金额':result.iloc[:,2],'平均金额':result.iloc[:,3],'VIP等级':result.iloc[:,4]})

    result.set_index('账户名',inplace=True)
    result = result.loc[result['数量']>10,:]
    df.set_index('账户名',inplace=True)
    print(result.sort_values('数量',ascending=False))

    # 写入数据
    if len(result.index)>0:
        fw = open('result2.txt','w')

        for name in result.index:
            fw.write(f'会员等级：{int(result.loc[name,"VIP等级"])}\n')
            fw.write(f'账号：{name}\n')
            fw.write(f'最大充值金额：{int(result.loc[name,"最大金额"])}\n')
            fw.write(f'平均充值金额：{int(result.loc[name,"平均金额"])}\n')
            fw.write(f'2小时内申请次数：{result.loc[name,"数量"]}\n')
            fw.write('-----------------------\n')
        fw.write(f'总计：{len(result.index)}')
        fw.close()
        with open('result2.txt','r') as fr:
            text = fr.read()
        bot_da = telebot.TeleBot('6106076754:AAHjxPSBpyjwpY-lq1iEslUufW46XQvAfr0') # -953042672
        #bot_m = telebot.TeleBot("6377312623:AAGz3ZSMVswWq0QVlihRPklw8b7skSBP16Y")
        # bot_a = telebot.TeleBot('6321364690:AAFvTiujKew0Fqi6OfL6awyM5Nx2LscJbVs')
        # bot_da.send_message(-677235937,text)   -996504819
        bot_da.send_message(-677235937,text)
        bot_da.stop_polling()
    else:
        bot_da = telebot.TeleBot('6106076754:AAHjxPSBpyjwpY-lq1iEslUufW46XQvAfr0')
        bot_da.send_message(6279115720,'充值拉单无数据')
        bot_da.stop_polling()
job()

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [19]:
import requests
import json
import pandas as pd
url = 'http://fundmng.bsportsadmin.com/api/manage/activity/user/vip/v2/modify/modifyRecord'
header = {
    'Device_id': '1.0',
    'Os_type': '0',
    'Sign': '525f286b21988b7c6a79f623f62f9695',
    'Timestamp': '1691886184000',
    'Token': 'e2488ea7cd074d1db8751f85c4fc01fb',
    'Uid': '690',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    'Version': '1.0'
}

session = requests.session()
vip_dic = {'调整时间':[],'会员账号':[],'VIP等级':[]}
for page in range(1,2606):
    print(f'第{page}页。。。')
    data = {
        'page': page,
        'size': 500,
        'modifyStatus': '-1',
        'modifyTimeStart': 1601568000000,
        'modifyTimeEnd': 1697039999999
    }
    response = session.post(url=url,data=data,headers=header)
    response.encoding = 'utf-8'

    obj = json.loads(response.text)

    for i in obj['data']['list']:
        vip_dic['调整时间'].append(i["modifyTime"])
        vip_dic['会员账号'].append(i['username'])
        vip_dic['VIP等级'].append(i["vipLevelAfter"])
    print('会员人数',len(vip_dic['会员账号']))
df = pd.DataFrame(vip_dic)
df.to_csv(r'C:\Users\User\IdeaProjects\project1\Bsport\充值拉单预警\vip2.csv',index=False)

第1页。。。
会员人数 500
第2页。。。
会员人数 1000
第3页。。。
会员人数 1500
第4页。。。
会员人数 2000
第5页。。。
会员人数 2500
第6页。。。
会员人数 3000
第7页。。。
会员人数 3500
第8页。。。
会员人数 4000
第9页。。。
会员人数 4500
第10页。。。
会员人数 5000
第11页。。。
会员人数 5500
第12页。。。
会员人数 6000
第13页。。。
会员人数 6500
第14页。。。
会员人数 7000
第15页。。。
会员人数 7500
第16页。。。
会员人数 8000
第17页。。。
会员人数 8500
第18页。。。
会员人数 9000
第19页。。。
会员人数 9500
第20页。。。
会员人数 10000
第21页。。。
会员人数 10500
第22页。。。
会员人数 11000
第23页。。。
会员人数 11500
第24页。。。
会员人数 12000
第25页。。。
会员人数 12500
第26页。。。
会员人数 13000
第27页。。。
会员人数 13500
第28页。。。
会员人数 14000
第29页。。。
会员人数 14500
第30页。。。
会员人数 15000
第31页。。。
会员人数 15500
第32页。。。
会员人数 16000
第33页。。。
会员人数 16500
第34页。。。
会员人数 17000
第35页。。。
会员人数 17500
第36页。。。
会员人数 18000
第37页。。。
会员人数 18500
第38页。。。
会员人数 19000
第39页。。。
会员人数 19500
第40页。。。
会员人数 20000
第41页。。。
会员人数 20500
第42页。。。
会员人数 21000
第43页。。。
会员人数 21500
第44页。。。
会员人数 22000
第45页。。。
会员人数 22500
第46页。。。
会员人数 23000
第47页。。。
会员人数 23500
第48页。。。
会员人数 24000
第49页。。。
会员人数 24500
第50页。。。
会员人数 25000
第51页。。。
会员人数 25500
第52页。。。
会员人数 26000
第53页。。。
会员人数 26500
第54页。。。
会员人数 27000
第55

In [3]:
df.head()

,会员账号,VIP等级
0,thdh445e5,1
1,Hoangphu626163,1
2,Hoanghoa89,1
3,clone267,1
4,ytt1999,1


In [4]:
df.to_csv(r'C:\Users\User\IdeaProjects\project1\Bsport\充值拉单预警\vip.csv',index=False)

In [8]:
import json
obj = json.loads(response.text)
vip_dic = {'会员账号':[],'VIP等级':[]}
for i in obj['data']['list']:
    vip_dic['会员账号'].append(i['username'])
    vip_dic['VIP等级'].append(i["vipLevelAfter"])
vip_dic

{'会员账号': ['w520221',
  'j02fw83',
  'Tranhiep1994',
  'Huybui1471',
  'hr736vp',
  'bbncuh00',
  'fgh5r5',
  'Dungbipem3636',
  'Hpnam2202',
  'nvt270205',
  'hui5r5',
  'Cuongca111',
  'Satnhom1368',
  'wf886886886',
  'g1525013',
  'Laivantu10',
  '0889917996to',
  '0889917996bmt',
  'giongbao1',
  'liuzd588528',
  'ljy777',
  'Nguyenminh1982',
  'shan961226',
  'Thaihung1997',
  'Luuthoainguyen9',
  'trinh12356',
  'moke400',
  'q373728',
  'ngocanh100990',
  'king1819',
  'vito123',
  'Hzx66666666',
  'Anhqui18788',
  'Manhvan87',
  'chrischeng1998',
  'Nhatkhanh87',
  'w0786321',
  'vophuoc123',
  'Thaotyt12356',
  'Botam12341',
  'Thachotaodi2903',
  'ysyy2023',
  'an666666',
  'j11940w',
  'dgz2450606',
  'wangji456',
  'nvsinh9999',
  'cs3nnsfq',
  'sm1989',
  'Phanhatinh2005',
  'aaa19687402',
  'Bom1234',
  'tselai1013',
  'qiuzhu2001',
  'letho1005',
  'Namduongka123',
  'quangdung16',
  'hanchelo1030',
  'Thinh1604',
  'thaituan7979',
  'lxg778899',
  'vne9999',
  'Cocain19

In [9]:
import pandas as pd
pd.DataFrame(vip_dic)

,会员账号,VIP等级
0,w520221,1
1,j02fw83,1
2,Tranhiep1994,1
3,Huybui1471,1
4,hr736vp,1
...,...,...
495,Huang778899,3
496,hjl77778888,3
497,Chilinh16262016,2
498,SamSamTV1,2


In [3]:
import time
import xlwings as xw
import jsonpath
import requests
import json
import pandas as pd
import datetime

# 替换为你的App Key和App Secret
app_key = "rCcSsGGfuu10lHFAPmBGsRunpp9wha09"
app_secret = "apNWQFX57OvLQvW7E5wyD8oWHdhyboqx"

# 构造请求URL
url = "https://openapi.baidu.com/oauth/2.0/token"
params = {
    "grant_type": "client_credentials",
    "client_id": app_key,
    "client_secret": app_secret
}

# 发送请求并获取access-token
response = requests.get(url, params=params)
data = response.json()
access_token = data["access_token"]
access_token
url_siteid = f'https://openapi.baidu.com/rest/2.0/tongji/config/getSiteList?access_token={access_token}'
response = requests.get(url_siteid)
response.encoding='utf-8'
response.text

'24.c64060120973f81c6c17e8e9bbb85c3b.2592000.1697002521.282335-38761358'

'{"error_code":11002,"error_message":"\\u53c2\\u6570\\uff1auser \\u672a\\u63d0\\u4f9b"}'

In [5]:
with open(r'C:\Users\User\Desktop\SEO\12-18\dic_website.txt','r') as f:
    dic_website=f.read()
dic_website


"{'dk.bty': 18173781, 'zed.bty': 18173898, 'dazhi.bty': 18173904, 'ben.bty': 18185202, 'bike.bty': 18576075, 'duobo.bty': 18617945, 'alen.bty': 18659719, 'peter.bty': 18808332, 'max.bty': 19459600, 'aber.bty': 19409688, 'paddy.com': 19409639, 'paddy.bty': 19409636, 'martin.com': 17650492, 'groot.com': 17650496, 'dk.com': 17650498, 'aber.com': 17650500, 'duobo.com': 17667703, 'tonyb.com': 18173377, 'ben.com': 18185145, 'leslie.com': 18276831, 'kiwi.com': 18317053, 'billy.com': 19230037, 'dazhi.com': 16226474, 'redquan.com': 16234023, 'zed.com': 17227146, 'hugo.com': 17607180, 'maxmile.com': 18173387}\n"

In [6]:
url_web = f'https://openapi.baidu.com/rest/2.0/tongji/report/getData?access_token={access_token}&site_id=18173781&method=overview/getTimeTrendRpt&start_date={20230911}&end_date={20230911}&metrics=pv_count,visitor_count,ip_count'
response = requests.get(url_web)
response.encoding='utf8'
response.text

'{"error_code":11002,"error_message":"\\u53c2\\u6570\\uff1auser \\u672a\\u63d0\\u4f9b"}'

In [4]:
url_siteid = f'https://openapi.baidu.com/rest/2.0/tongji/config/getSiteList?access_token={access_token}'
response = requests.get(url_siteid)
response.status_code

200